# DOGS ADOPTIONS PROJECT

**Importing libraries**

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 50)
pd.options.display.max_columns = 40
pd.options.display.max_rows = 700

**Importing the 'dogs', 'dog_travel' and 'NST_EST' dataframe:**

In [3]:
dogs = pd.read_csv("adoptions/dogs.csv")

In [4]:
dog_travel = pd.read_csv("adoptions/dogTravel.csv")

In [5]:
NST_EST = pd.read_csv("adoptions/NST-EST2021-POP.csv", header=None)
NST_EST.columns = ['state', 'population']

**The str.replace() method replaces any dots in the population column with an empty string, and then the astype() method converts the resulting strings to integers**

In [6]:
NST_EST['population'] = NST_EST['population'].str.replace('.', '', regex=False).astype(int)

**Importing the 'states' dataset, which contains all the USA states and their abbreviation (this dataset will be used in exercise 5 and 9)**

In [7]:
states = pd.read_csv("adoptions/states.csv", sep=';')

states['zip_end'] = states['zip_end'].fillna(0)
states['zip_end'] = states['zip_end'].astype(int)

states['zip_start'] = states['zip_end'].fillna(0)
states['zip_start'] = states['zip_end'].astype(int)

states.head()

KeyError: 'zip_end'

# PRE-PROCESSING

**Cleaning the 'name' column in the 'dogs' dataframe**

**In the following rows, column 'status' contains the dog name in a "dirty" form and also what should be contained in the 'description' column**

In [ ]:
names_and_desc = dogs[dogs['status'] != 'adoptable']['name'] 
names_and_desc.head()

**Isolate the names in "dirty" form in a separate list:**

In [ ]:
dirty_names = []
for name_desc in names_and_desc:
    dirty_names.append(name_desc.split(sep=",")[0])
dirty_names

There are two pattern in the diry names: 
1. name \\\nickname-char\\\\"
2. \\\nickname-char\\\\"

in the first case we want to isolate the name

in the second case we isolate the nickname

In [ ]:
nomi = []
for name in dirty_names:
    match = re.split(r'[:\\]', name)
    if match[0] == '':                  #pattern n° 2 -> the 'match' list contains only the nickname
        nomi.append(match[1].strip())   #remove leading or trailing spaces with 'strip()' and appending to the 'nomi' list
    else:                               #pattern n° 1 -> the 'match' list contains only the nickname
        nomi.append(match[0].strip()) 
dogs_clean = dogs.copy(deep=True)       #create a copy of the 'dogs' dataframe ('dogs_clean') that is completely 
                                        #indipendent from it (thanks to deep=True): if you modify 'dogs' this will
                                        # not affect 'dogs_clean'

dogs_clean.loc[dogs_clean['status'] != 'adoptable', 'name'] = nomi 
# ^ assign the cleaned names in the column 'name' of dogs_clean

dogs_clean[dogs_clean['status'] != 'adoptable']['name']

**Shift the columns corresponding to the previous names from 'status' to 'accessed' of one to the right**

In [ ]:
dogs_clean.loc[dogs_clean['status'] != 'adoptable','status':'accessed'] = dogs_clean.loc[dogs_clean['status'] != 'adoptable','status':'accessed'].shift(1, axis=1)
dogs_clean.loc[dogs_clean['status'] != 'adoptable']

In [ ]:
dogs_clean.loc[17610:17630,'status':'accessed']

# 1. Extract all dogs with status that is not adoptable

**Dataset before cleaning:**

In [ ]:
dogs[dogs['status'] != 'adoptable'][['id', 'name', 'status']].head()

**Cleaned dataset:**

In [ ]:
not_ad_dogs = dogs_clean[dogs_clean['status'] != 'adoptable'][['id', 'name', 'status']]
not_ad_dogs.head()

**Dataset before cleaning:**

In [ ]:
dogs[dogs['status'] != 'adoptable'][['status']].head()

**Cleaned dataset:**

In [ ]:
dogs_clean[dogs_clean['status'] != 'adoptable'][['status']].head()

# 2. For each (primary) breed, determine the number of dogs

In [ ]:
breed_counts = dogs.groupby('breed_primary').count()[['id']].rename(columns={'id':'counts'})#.reset_index()[['breed_primary','counts']]
breed_counts

# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed

In [ ]:
dogs_clean.head()

**METHOD 1:**

**Considering the 'breed_secondary' column: if it contains the text 'Mixed Breed' it means that the dog is of mixed breed, otherwise it is not**

**Selecting the 'Mixed Breed' dogs**:

In [ ]:
mixed_for_breed = dogs_clean[dogs_clean['breed_secondary'] == 'Mixed Breed'].groupby('breed_primary', as_index=False).count()
mixed_for_breed['n_mixed'] = mixed_for_breed['id']
mixed_for_breed = mixed_for_breed[['breed_primary', 'n_mixed']]
mixed_for_breed.head()

**Selecting the 'Not of Mixed Breed' dogs**:

In [ ]:
not_mixed_for_breed = dogs_clean[dogs_clean['breed_secondary'] != 'Mixed Breed'].groupby('breed_primary', as_index=False).count()
not_mixed_for_breed['n_not_mixed'] = not_mixed_for_breed['id']
not_mixed_for_breed = not_mixed_for_breed[['breed_primary', 'n_not_mixed']]
not_mixed_for_breed

**Merging the tables**:

In [ ]:
ratio_mixed = pd.merge(mixed_for_breed, not_mixed_for_breed, on='breed_primary')
ratio_mixed

**Calculating the ratio between 'mixed' and 'not_mixed'**:

In [ ]:
ratio_mixed['ratio_M_notM'] = ratio_mixed['n_mixed']/not_mixed_for_breed['n_not_mixed']
ratio_mixed

**METHOD 2:**

**Considering the 'breed_mixed' column**

**If True it means that the dog is of mixed breed**

In [ ]:
mixed_breed = dogs_clean.where(dogs_clean['breed_mixed']== True).groupby('breed_primary').size().reset_index(name='mixed')
mixed_breed.head()

**If False it means that the dog is not of mixed breed**

In [ ]:
not_mixed_breed = dogs.where(dogs['breed_mixed'] == False).groupby('breed_primary').size().reset_index(name='not_mixed')
not_mixed_breed.head()

**We merge and calculate the ratio**

In [ ]:
breed_ratio = pd.merge(mixed_breed, not_mixed_breed, on='breed_primary')
breed_ratio.head()

In [ ]:
breed_ratio['ratio'] = breed_ratio['mixed'] / breed_ratio['not_mixed']
breed_ratio.head()


# 4. For each (primary) breed, determine the earliest and the latest posted timestamp.


**We check and convert the 'posted' type to datetime**

In [ ]:
print(dogs_clean['posted'].dtypes)

In [ ]:
dogs_clean['posted'] = pd.to_datetime(dogs_clean['posted'])

In [ ]:
dogs_clean.dropna(subset=['posted'], inplace=True)
len(dogs_clean)

In [ ]:
print(dogs_clean['posted'].dtypes)

**We find the min and the max for each 'breed_primary' and merge them in a single table**

In [ ]:
earliest_timestamps = dogs_clean.groupby(by='breed_primary')['posted'].min().reset_index()
latest_timestamps = dogs_clean.groupby(by='breed_primary')['posted'].max().reset_index()

In [ ]:
timestamps = pd.merge(earliest_timestamps, latest_timestamps, on="breed_primary").rename(columns={'posted_x':'earliest_timestamp', 'posted_y':'latest_timestamp'})
timestamps.head()

# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

**We count the values of Females and Males for each State and then calculate the sex imbalance**

In [ ]:
sex_counts = dogs_clean.groupby('contact_state')['sex'].value_counts().unstack()
sex_counts.head()

In [ ]:
sex_counts['sex_imbalance'] = sex_counts['Male'] - sex_counts['Female']
sex_counts.head()

**Using the 'states' df, we find the full name of the state with the largest sex imbalance, through the 'abbreviation' column**

In [ ]:
max_imbalance_state = sex_counts['sex_imbalance'].idxmax()
max_imbalance_state_name = states.loc[states['abbreviation'] == max_imbalance_state].values[0][0]
print(f'The state with the largest sex imbalance is: {max_imbalance_state_name}')

# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [ ]:
mean_stay_cost = dogs_clean.groupby(['age', 'size'], as_index=False)[['age','size','stay_duration','stay_cost']].mean(['stay_duration','stay_cost'])
mean_stay_cost

**Apply a categorization to the values contained in the 'age' column:**

In [ ]:
mean_stay_cost['age'] = pd.Categorical(mean_stay_cost['age'], 
                                       categories=['Baby', 'Young', 'Adult', 'Senior'],
                                       ordered=True
                                      )

mean_stay_cost['size'] = pd.Categorical(mean_stay_cost['size'], 
                                        categories = ['Small', 'Medium', 'Large', 'Extra Large'],
                                        ordered=True
                                       )

**Renaming the columns:**

In [ ]:
mean_stay_cost.rename(columns={"stay_duration": "mean_stay_duration", "stay_cost": "mean_stay_cost"}, inplace=True)

In [ ]:
mean_stay_cost.groupby(by=['age', 'size'])[['mean_stay_duration', 'mean_stay_cost']].mean() 
#use 'age' and 'size' as index and sorting

# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [ ]:
dog_travel.head()

**We count the number of travels for each 'id' and select only the ones >= 3**

In [ ]:
n_viaggi = dog_travel.groupby('id').count()
n_viaggi.rename(columns={'index' : 'n_travels'}, inplace=True)
n_viaggi = n_viaggi[['n_travels']]
n_viaggi.head()

In [ ]:
n_viaggi = n_viaggi[n_viaggi['n_travels'] >= 3]
n_viaggi.head()

**Merging with the 'dogs_clean' df, we can find the 'breed_primary'**

In [ ]:
atleast_3 = pd.merge(n_viaggi, dogs_clean, on='id')
atleast_3[['id', 'n_travels', 'breed_primary']].head()

In [ ]:
len(atleast_3)

# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [ ]:
dog_travel.head()

In [ ]:
"""
The function takes two strings in input 'found' and 'manual'
and returns 'found' if 'manual' is Null
returns 'manual' otherwise
"""
def correct_state(found, manual):
    if pd.isna(manual):
        return found
    else:
        return manual

**Apply the above function to every row of the 'dog_travel' dataframe using the lambda row sintax and storing the result in a list called 'lista'**

In [ ]:
lista = dog_travel.apply(lambda row: correct_state(row['found'], row['manual']), axis=1)
lista

**Replacing the values in 'lista'**

In [ ]:
dog_travel['found'] = lista
dog_travel.head()

# 9. For each state, compute the ratio between the number of travels and the population.

In [ ]:
dog_travel.head()

**We merge the 'NST_EST' df and the 'states' df to have a complete df with 'abbreviation', 'state', 'population'**

In [ ]:
NST = pd.merge(NST_EST, states, on='state')
NST[['abbreviation', 'state', 'population']].head()

**We count the number of travels for each state**

In [ ]:
travels_count = dog_travel.groupby('contact_state').size().reset_index(name='travels')
travels_count.head()

**We notice that '17325' correspond to the zip code of Pennsylvania (PA), so we replace it and run the count again**

In [ ]:
dog_travel['contact_state'].replace('17325', 'PA', inplace=True)

In [ ]:
travels_count = dog_travel.groupby('contact_state').size().reset_index(name='travels')
travels_count

**We merge as to have a df with 'contact_state', 'population', 'travels' so now we can calculate the ratio in a new column**

In [ ]:
state_ratio = pd.merge(NST, travels_count, left_on='abbreviation', right_on='contact_state')[['contact_state','population', 'travels']]
state_ratio.head()

In [ ]:
state_ratio['ratio'] = state_ratio['travels'] / state_ratio['population']
state_ratio.rename(columns = {'contact_state' : 'state'}, inplace = True)
state_ratio.head()

# 10. For each dog, compute the number of days from the posted day to the day of last access.


**Control data types of 'posted' and 'accessed' columns and changing them to 'datetime' if they are not**

In [ ]:
print(dogs_clean['posted'].dtypes)

In [ ]:
print(dogs_clean['accessed'].dtypes)

In [ ]:
dogs_clean['accessed'] = pd.to_datetime(dogs_clean['accessed'], errors = 'coerce')

In [ ]:
dogs.dropna(subset=['accessed'], inplace=True)
len(dogs_clean)

In [ ]:
print(dogs_clean['posted'].dtypes)

In [ ]:
print(dogs_clean['accessed'].dtypes)

**Converting from a 'datetime' to a 'date'**

In [ ]:
dogs_clean['posted'] = dogs_clean['posted'].dt.date

In [ ]:
dogs_clean['accessed'] = dogs_clean['accessed'].dt.date

**Calculating the days between 'posted' and 'accessed'**

In [ ]:
dogs_clean['days_between'] = (dogs_clean['accessed'] - dogs_clean['posted']).dt.days

In [ ]:
days_between = dogs_clean[['id', 'posted', 'accessed', 'days_between']]
days_between

# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

**We devide the days that have passed by 7, as to have the number of weeks**

In [ ]:
dogs_clean['weeks_between'] = (dogs_clean['accessed'] - dogs_clean['posted']).dt.days/7
dogs_clean['weeks_between'] = dogs_clean['weeks_between'].round(2)

**We then bin the number of weeks into pre-defined ranges, and save to a new column**

In [ ]:
dogs_clean['weeks_range'] = pd.cut(dogs_clean['weeks_between'], bins=[-np.inf, 1, 4, 8, 12, 16, 24, 25, 52, np.inf], labels=['< 1 week', '1-4 weeks', '4-8 weeks', '8-12 weeks', '12-16 weeks', '16 weeks - 6 months', '> 6 months', '1 year', '> 1 year'])

In [ ]:
weeks_between = dogs_clean[['id', 'posted', 'accessed', 'days_between', 'weeks_between', 'weeks_range']]
weeks_between

# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [ ]:
# Drop any rows with missing values
dogs_sub = dogs_clean.dropna(subset=['breed_primary', 'description', 'sex'])
dogs_sub

In [ ]:
# Preprocess the description field
def preprocess_description(desc):
    if pd.isna(desc):
        return []
    desc = re.sub(r'\W+', ' ', desc.lower())
    return desc.split()

^ this function takes a string as input and returns a list of preprocessed words. The preprocessing steps include converting the string to lowercase, removing any non-alphanumeric characters, and splitting the string into a list of words.

In [ ]:
# Define a function to calculate the percentage of shared words between two descriptions

def shared_word_percentage(desc1, desc2):
    set1 = set(desc1)
    set2 = set(desc2)
    if len(set1) == 0 or len(set2) == 0:
        return 0.0
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

^ this function is defined to calculate the percentage of shared words between two descriptions. This is done by first converting the descriptions to sets of words, and then calculating the intersection and union of the sets.


In [ ]:
dogs_sub.loc[:, 'description_processed'] = dogs_sub['description'].apply(preprocess_description)

^ the 'description_processed' column in the dogs_clean DataFrame is created by applying the 'preprocess_description' function to the 'description' column using the 'apply()' method.

In [ ]:
# Create a dictionary to store the processed descriptions and their corresponding IDs
desc_dict = dict(zip(dogs_sub['id'], dogs_sub['description_processed']))
#desc_dict

^ a dictionary called 'desc_dict' is created to store the processed descriptions and their corresponding IDs.

In [ ]:
# Use a hash table to store the processed descriptions and their corresponding IDs
desc_hash = {}
for i, desc in enumerate(dogs_sub['description_processed']):
    desc_hash[hash(' '.join(desc))] = desc_hash.get(hash(' '.join(desc)), []) + [dogs_sub.iloc[i]['id']]

^ a hash table called 'desc_hash' is created to store the processed descriptions and their corresponding IDs. The hash table is constructed by iterating over the 'description_processed' column in the dogs_sub DataFrame, and using the 'hash()' function to generate a hash value for each description. The hash value is used as a key in the desc_hash dictionary, and the corresponding ID is added to a list of IDs associated with that hash value.

In [ ]:
# Find the duplicate pairs using the hash table
duplicate_pairs = []
for i, desc in enumerate(dogs_sub['description_processed']):
    desc_hash[hash(' '.join(desc))].remove(dogs_sub.iloc[i]['id'])
    for j in desc_hash[hash(' '.join(desc))]:
        if dogs_sub.iloc[i]['breed_primary'] == dogs_sub[dogs_sub['id'] == j]['breed_primary'].values[0] and \
           dogs_sub.iloc[i]['breed_secondary'] == dogs_sub[dogs_sub['id'] == j]['breed_secondary'].values[0] and \
           dogs_sub.iloc[i]['sex'] == dogs_sub[dogs_sub['id'] == j]['sex'].values[0]:
            shared_words = shared_word_percentage(desc, desc_dict[j])
            if shared_words >= 0.9:
                duplicate_pairs.append((dogs_sub.iloc[i]['id'], j))
    desc_hash[hash(' '.join(desc))].append(dogs_sub.iloc[i]['id'])

^ a loop is used to iterate over the 'description_processed' column in the dogs_sub DataFrame. For each record, the ID is removed from the list of IDs associated with its hash value in the desc_hash dictionary. Then, a nested loop is used to compare the current record to all other records in the desc_hash dictionary with the same hash value. If the current record and another record have the same breeds and sex, and their shared word percentage is at least 90%, then they are considered a duplicate pair and their IDs are added to the duplicate_pairs list.

^^ execution time: ~ 1 minute

**We save the pairs in a new df and populate with the corresponding values**

In [ ]:
df_duplicate_pairs = pd.DataFrame(duplicate_pairs, columns=['id_1', 'id_2'])
df_duplicate_pairs.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = 40

In [ ]:
dogs_sub[dogs_sub['id'] == 44696946][['id', 'breed_primary', 'breed_secondary', 'sex', 'description']]

In [ ]:
dogs_sub[dogs_sub['id'] == 45301676][['id', 'breed_primary', 'breed_secondary', 'sex', 'description']]

In [ ]:
# Merge 'df_duplicate_pairs' with 'dogs' to populate it with the corresponding values
df_duplicate_pairs = pd.merge(df_duplicate_pairs, dogs, left_on='id_1', right_on='id')
df_duplicate_pairs = df_duplicate_pairs[['id_1', 'id_2', 'breed_primary', 'breed_secondary', 'sex', 'description']]
df_duplicate_pairs = pd.merge(df_duplicate_pairs, dogs, suffixes=['_1', '_2'], left_on='id_2', right_on='id')
df_duplicate_pairs = df_duplicate_pairs[['id_1', 'id_2', 'breed_primary_1', 'breed_primary_2', 'breed_secondary_1', 'breed_secondary_2', 'sex_1', 'sex_2', 'description_1', 'description_2']]

# Print the resulting DataFrame
df_duplicate_pairs.head()